In [107]:
import pandas as pd
import numpy as np
import os
from datetime import date
from da_utils.client.bigquery_client import BigQueryClient
from da_utils.client.google_sheets_client import GoogleSheetsClient
from da_utils.repository.google_sheets.google_sheets_repository import GoogleSheetsRepository
from da_utils.repository.bigquery.bigquery_repository import BigqueryRepository

from da_utils.client.slack_client import SlackClient
from da_utils.repository.slack.slack_repository import SlackRepository
pd.set_option("display.max_column", 100)

In [108]:
adc_path = '/home/linhnguyen/.config/gcloud/application_default_credentials.json'
bq_client = BigQueryClient(billing_project_id='prj-ts-p-analytic-8057', adc_file_path_str=adc_path)
bq_repo = BigqueryRepository(bq_client)

bq_client_avay = BigQueryClient(billing_project_id='avay-a9925', adc_file_path_str=adc_path)
bq_repo_avay = BigqueryRepository(bq_client_avay)

client = GoogleSheetsClient()
repo = GoogleSheetsRepository(client)

slack_client = SlackClient(token=os.environ.get("SLACK_BOT_AUTHEN"))
slack_repo = SlackRepository(client=slack_client)

/home/linhnguyen/.local/lib/python3.10/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/linhnguyen/.local/lib/python3.10/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [109]:
df1 = pd.read_csv('/home/linhnguyen/Neo_datareport_campaign_keyword.csv')
df2 = pd.read_csv('/home/linhnguyen/Neo_datareport_campaign.csv')
df3 = pd.read_csv('/home/linhnguyen/Evo_datareport_campaign_keyword.csv')
df4 = pd.read_csv('/home/linhnguyen/Evo_datareport_campaign.csv')

df1['mkt_channel'] = '8131246892'
df2['mkt_channel'] = '8131246892'
df3['mkt_channel'] = '3154428833'
df4['mkt_channel'] = '3154428833'

df1.rename(columns = {'Day': 'date', 'Campaign ID': 'campaignid', 'Ad group ID':'adgroupid',
                      'Clicks':'clicks', 'Impressions':'impressions', 'Cost':'cost', 'Search keyword': 'criteria'}, inplace = True)
df2.rename(columns = {'Day': 'date', 'Campaign ID': 'campaignid', 'Ad group ID':'adgroupid',
                      'Clicks':'clicks', 'Impressions':'impressions', 'Cost':'cost'}, inplace = True)
df3.rename(columns = {'Day': 'date', 'Campaign ID': 'campaignid', 'Ad group ID':'adgroupid',
                      'Clicks':'clicks', 'Impressions':'impressions', 'Cost':'cost', 'Search keyword': 'criteria'}, inplace = True)
df4.rename(columns = {'Day': 'date', 'Campaign ID': 'campaignid', 'Ad group ID':'adgroupid',
                      'Clicks':'clicks', 'Impressions':'impressions', 'Cost':'cost'}, inplace = True)

df1['date'] = pd.to_datetime(df1['date']).apply(lambda x: x.strftime('%Y-%m-%d'))
df2['date'] = pd.to_datetime(df2['date']).apply(lambda x: x.strftime('%Y-%m-%d'))
df3['date'] = pd.to_datetime(df3['date']).apply(lambda x: x.strftime('%Y-%m-%d'))
df4['date'] = pd.to_datetime(df4['date']).apply(lambda x: x.strftime('%Y-%m-%d'))

df1['date'] = df1['date'].astype('dbdate')
df2['date'] = df2['date'].astype('dbdate')
df3['date'] = df3['date'].astype('dbdate')
df4['date'] = df4['date'].astype('dbdate')

In [143]:
df2 = df2[['date','mkt_channel', 'campaignid', 'adgroupid', 'clicks', 'impressions', 'cost']]
df4 = df4[['date','mkt_channel', 'campaignid', 'adgroupid', 'clicks', 'impressions', 'cost']]
total_camp = pd.concat([df2,df4])

total_camp_no_adgroup = total_camp.groupby(['date','mkt_channel', 'campaignid']).agg({'clicks':'sum', 'impressions':'sum', 'cost':'sum'}).reset_index()

total_camp
# total_camp[(total_camp['campaignid'] == 18281067979)&(total_camp['adgroupid'] == 142431701278)
#           &(total_camp['date'] == date(2022,10,1))]


,date,mkt_channel,campaignid,adgroupid,clicks,impressions,cost
0,2023-02-20,8131246892,19495146235,145869229358,1,6,0.17
1,2023-02-19,8131246892,19495146235,145869229358,1,12,0.02
2,2023-02-18,8131246892,19495146235,145869229358,1,13,0.02
3,2023-02-17,8131246892,19495146235,145869229358,5,29,0.14
4,2023-02-16,8131246892,19495146235,145869229358,1,20,0.02
...,...,...,...,...,...,...,...
6591,2022-09-09,3154428833,18279483591,143536638840,2,26,0.40
6592,2022-09-09,3154428833,18279483591,146621274848,12,597,2.28
6593,2022-09-09,3154428833,18279483591,146621274888,7,94,1.26
6594,2022-09-08,3154428833,18271685755,141908709798,105,530,8.14


In [138]:
df5 = pd.read_csv('/home/linhnguyen/EVO_onlycampaign.csv')
df6 = pd.read_csv('/home/linhnguyen/NEO_onlycampaign.csv')
df5.rename(columns = {'Day': 'date', 'Campaign ID': 'campaignid',
                      'Clicks':'clicks', 'Impressions':'impressions', 'Cost':'cost'}, inplace = True)
df6.rename(columns = {'Day': 'date', 'Campaign ID': 'campaignid',
                      'Clicks':'clicks', 'Impressions':'impressions', 'Cost':'cost'}, inplace = True)

df5['date'] = pd.to_datetime(df5['date']).apply(lambda x: x.strftime('%Y-%m-%d'))
df6['date'] = pd.to_datetime(df6['date']).apply(lambda x: x.strftime('%Y-%m-%d'))

df5['date'] = df5['date'].astype('dbdate')
df6['date'] = df6['date'].astype('dbdate')

df6['mkt_channel'] = '8131246892'
df5['mkt_channel'] = '3154428833'

df5[df5['campaignid'] == 19656700754]


,date,Campaign,campaignid,cost,clicks,impressions,mkt_channel
239,2023-02-15,PM_vOTP,19656700754,134.438180,943,438188,3154428833
254,2023-02-14,PM_vOTP,19656700754,358.333482,2257,1237946,3154428833
264,2023-02-13,PM_vOTP,19656700754,562.237442,5403,1359941,3154428833
273,2023-02-12,PM_vOTP,19656700754,623.998270,13103,605170,3154428833
287,2023-02-11,PM_vOTP,19656700754,271.075129,4978,254036,3154428833
293,2023-02-10,PM_vOTP,19656700754,63.891034,410,10586,3154428833
300,2023-02-09,PM_vOTP,19656700754,301.313242,5009,175700,3154428833
318,2023-02-08,PM_vOTP,19656700754,22.747660,390,71980,3154428833


In [145]:
df5 = df5[['date','mkt_channel', 'campaignid', 'clicks', 'impressions', 'cost']]
df6 = df6[['date','mkt_channel', 'campaignid', 'clicks', 'impressions', 'cost']]
only_camp = pd.concat([df5,df6])

no_adgroup = only_camp.merge(total_camp_no_adgroup, how = 'left', on = ['date','mkt_channel', 'campaignid'] 
                ,suffixes = ('_total', '_adgroup')).assign(
                                            cost = lambda x: np.where(x.cost_adgroup.isnull(), x.cost_total,x.cost_total - x.cost_adgroup),
                                            clicks = lambda x: np.where(x.clicks_adgroup.isnull(), x.clicks_total, x.clicks_total - x.clicks_adgroup),
                                            impressions = lambda x: np.where(x.impressions_adgroup.isnull(), x.impressions_total, x.impressions_total- x.impressions_adgroup),
)
no_adgroup = no_adgroup[no_adgroup['impressions']> 0]

no_adgroup[no_adgroup['date'] >= date(2023,2,8)]
no_adgroup['adgroupid'] = 0
no_adgroup = no_adgroup[['date','mkt_channel', 'campaignid', 'adgroupid', 'clicks', 'impressions', 'cost']]
total_camp = pd.concat([no_adgroup, total_camp])
total_camp[total_camp['adgroupid']==0]

,date,mkt_channel,campaignid,adgroupid,clicks,impressions,cost
132,2023-03-02,3154428833,19677759723,0,19.0,3655.0,3.712490
134,2023-03-01,3154428833,19677759723,0,76.0,6787.0,8.509280
144,2023-02-28,3154428833,19677759723,0,24.0,769.0,4.997150
146,2023-02-27,3154428833,19677759723,0,30.0,198.0,2.848565
153,2023-02-26,3154428833,19677759723,0,22.0,301.0,2.173482
159,2023-02-25,3154428833,19677759723,0,35.0,341.0,2.935910
165,2023-02-24,3154428833,19677759723,0,48.0,374.0,3.585305
175,2023-02-23,3154428833,19677759723,0,60.0,517.0,4.068965
176,2023-02-22,3154428833,19677759723,0,61.0,345.0,4.330113
182,2023-02-21,3154428833,19677759723,0,72.0,447.0,5.191915


In [146]:
df1_search= df1.groupby(['date','mkt_channel', 'campaignid', 'adgroupid']).agg({'clicks':'sum', 'impressions':'sum', 'cost':'sum'}).reset_index()
df3_search= df3.groupby(['date','mkt_channel', 'campaignid', 'adgroupid']).agg({'clicks':'sum', 'impressions':'sum', 'cost':'sum'}).reset_index()
search_camp = pd.concat([df1_search, df3_search])

search_camp

search_camp['date'] = search_camp['date'].astype('dbdate')
search_camp[(search_camp['campaignid'] == 18281067979)&(search_camp['adgroupid'] == 142431701278)
          &(search_camp['date'] == date(2022,10,1))]


,date,mkt_channel,campaignid,adgroupid,clicks,impressions,cost
530,2022-10-01,3154428833,18281067979,142431701278,33,421,9.13


In [147]:
no_keyword = total_camp.merge(search_camp, how = 'left', on = ['date', 'mkt_channel', 'campaignid', 'adgroupid'], 
                                        suffixes = ('_total', '_search')).assign(
                                            cost = lambda x: np.where(x.cost_search.isnull(), x.cost_total,x.cost_total - x.cost_search),
                                            clicks = lambda x: np.where(x.clicks_search.isnull(), x.clicks_total, x.clicks_total - x.clicks_search),
                                            impressions = lambda x: np.where(x.impressions_search.isnull(), x.impressions_total, x.impressions_total- x.impressions_search),
)

no_keyword = no_keyword[no_keyword['impressions_total'] - no_keyword['impressions_search']!=0]
no_keyword['criteria'] = 'No keyword'
no_keyword = no_keyword[['date','mkt_channel', 'campaignid', 'adgroupid', 'criteria', 'clicks', 'impressions', 'cost']]

no_keyword[(no_keyword['mkt_channel'] == '3154428833') &(no_keyword['date'] == date(2022,10,1))]

,date,mkt_channel,campaignid,adgroupid,criteria,clicks,impressions,cost
6560,2022-10-01,3154428833,18279483591,143536638840,No keyword,0.0,1.0,0.000000
6561,2022-10-01,3154428833,18279483591,138320158262,No keyword,0.0,5.0,0.000000
6563,2022-10-01,3154428833,18279483591,139731572566,No keyword,0.0,2.0,0.000000
6566,2022-10-01,3154428833,18279483591,146621275368,No keyword,0.0,21.0,0.000000
6567,2022-10-01,3154428833,18279483591,146621275128,No keyword,0.0,2.0,0.000000
6569,2022-10-01,3154428833,18270835165,144553723561,No keyword,214.0,8675.0,80.205481
6572,2022-10-01,3154428833,18279483591,146621275328,No keyword,0.0,1.0,0.000000
6573,2022-10-01,3154428833,18279483591,149529019588,No keyword,0.0,2.0,0.000000
6575,2022-10-01,3154428833,18279483591,146621274888,No keyword,0.0,12.0,0.000000
6577,2022-10-01,3154428833,18279483591,146621274928,No keyword,1.0,91.0,0.216177


In [148]:
df1 = df1[['date','mkt_channel', 'campaignid', 'adgroupid', 'criteria', 'clicks', 'impressions', 'cost']]
df3 = df3[['date','mkt_channel', 'campaignid', 'adgroupid', 'criteria', 'clicks', 'impressions', 'cost']]
keyword = pd.concat([df1,df3])
keyword

,date,mkt_channel,campaignid,adgroupid,criteria,clicks,impressions,cost
0,2023-02-20,8131246892,19495146235,145869229358,tpbank evo hoanghamobile,1,1,0.17
1,2023-02-20,8131246892,19495146235,145869229358,hoàngha,0,1,0.00
2,2023-02-20,8131246892,19495146235,145869229358,hoanghamobile khuyến mãi,0,1,0.00
3,2023-02-20,8131246892,19495146235,145869229358,hoanghamobile,0,2,0.00
4,2023-02-20,8131246892,19495146235,145869229358,hoànghamobile,0,1,0.00
...,...,...,...,...,...,...,...,...
272336,2022-09-07,3154428833,18270835165,144553723561,tpbank evo,0,0,0.00
272337,2022-09-07,3154428833,18270835165,144553723561,tpbank evo là gì,0,0,0.00
272338,2022-09-07,3154428833,18270835165,144553723561,tpbankevo,0,0,0.00
272339,2022-09-07,3154428833,18270835165,144553723561,vpbank evo,0,0,0.00


In [149]:
ggads_before_april = pd.concat([no_keyword, keyword])
ggads_before_april['campaignid'] = ggads_before_april['campaignid'].astype(str)
ggads_before_april['adgroupid'] = ggads_before_april['adgroupid'].astype(str)
ggads_before_april['date'] = ggads_before_april['date'].astype('dbdate')
ggads_before_april[ggads_before_april['campaignid']== '19677759723']

,date,mkt_channel,campaignid,adgroupid,criteria,clicks,impressions,cost
0,2023-03-02,3154428833,19677759723,0,No keyword,19.0,3655.0,3.712490
1,2023-03-01,3154428833,19677759723,0,No keyword,76.0,6787.0,8.509280
2,2023-02-28,3154428833,19677759723,0,No keyword,24.0,769.0,4.997150
3,2023-02-27,3154428833,19677759723,0,No keyword,30.0,198.0,2.848565
4,2023-02-26,3154428833,19677759723,0,No keyword,22.0,301.0,2.173482
5,2023-02-25,3154428833,19677759723,0,No keyword,35.0,341.0,2.935910
6,2023-02-24,3154428833,19677759723,0,No keyword,48.0,374.0,3.585305
7,2023-02-23,3154428833,19677759723,0,No keyword,60.0,517.0,4.068965
8,2023-02-22,3154428833,19677759723,0,No keyword,61.0,345.0,4.330113
9,2023-02-21,3154428833,19677759723,0,No keyword,72.0,447.0,5.191915


In [150]:
ggads_before_april = ggads_before_april.drop(columns = ['mkt_channel'], axis = 1)
ggads_before_april[(ggads_before_april['campaignid']== '18281067979')
                   &(ggads_before_april['date']== date(2022,10,1))
                  &(ggads_before_april['adgroupid']== '142431701278')]


,date,campaignid,adgroupid,criteria,clicks,impressions,cost
250013,2022-10-01,18281067979,142431701278,jcb sacombank,0.0,2.0,0.00
250014,2022-10-01,18281067979,142431701278,làm thẻ jcb,0.0,16.0,0.00
250015,2022-10-01,18281067979,142431701278,thẻ JCB,3.0,28.0,0.47
250016,2022-10-01,18281067979,142431701278,thẻ jcb acb,1.0,1.0,0.29
250017,2022-10-01,18281067979,142431701278,thẻ jcb mb,1.0,1.0,0.16
250018,2022-10-01,18281067979,142431701278,thẻ jcb techcombank,24.0,295.0,7.43
250019,2022-10-01,18281067979,142431701278,thẻ jcb tpbank,1.0,8.0,0.15
250020,2022-10-01,18281067979,142431701278,thẻ jcb vietcombank,0.0,1.0,0.00
250021,2022-10-01,18281067979,142431701278,thẻ jcb vietinbank,0.0,2.0,0.00
250022,2022-10-01,18281067979,142431701278,thẻ tín dụng jcb,2.0,49.0,0.43


In [ ]:
ggads_before_april.to_csv('/home/linhnguyen/02.EVO/01.MKT_Cost/ggads_before_april.csv', index=False)